<a href="https://colab.research.google.com/github/McSon2/tiktoktest/blob/main/vid2cleantxt_minimal_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# vid2cleantxt

A basic example of how to use the [vid2cleantxt](https://github.com/pszemraj/vid2cleantxt) repo. 


Use this as a base to start your own customized notebook by `File > Save a copy' and adjust it for your use case, i.e. add Google Drive integration, etc.

---


## setup


In [ ]:
#@markdown add auto-Colab formatting with `IPython.display`
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

In [ ]:
#@markdown setup logging
import logging
from pathlib import Path
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
_log_location = Path.cwd() / "LOGFILE_vid2cleantxt_transcriber.log"

logging.basicConfig(
    level=logging.INFO,
    filename=_log_location,
    filemode='a',
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%m/%d/%Y %I:%M:%S",
)
print(f'logfile is at:\n\n{_log_location}')

logfile is at:

/content/LOGFILE_vid2cleantxt_transcriber.log


In [ ]:
!nvidia-smi

Wed Jan 25 19:03:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%time
pip install git+https://github.com/pszemraj/vid2cleantxt.git -q 


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 KB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## import & download source video



This demo downloads a **video file** with `wget` to the local runtime and transcribes it. If you'd like to use this for youtube transcription, you can copy/paste the following code to download a video with [yt-dlp](https://github.com/yt-dlp/yt-dlp)  :



```python
!pip install -U -q yt-dlp
my_yt_link = "https://youtu.be/_UiL7ErcPf0"
!yt-dlp $my_yt_link --recode-video mp4
# video saved to notebook cwd, ready to run vid2cleantxt
```

Make sure that you run the youtube download code **before** running `vid2cleantxt.transcribe.transcribe_dir()`


---


In [ ]:
#@markdown enter a downloadable URL to a video
vid_file_url = "https://www.dropbox.com/s/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4?dl=1" #@param {type:"string"}

import re

def extract_filename(url:str):
    base_name = re.findall(r'[^/]+$', url)[0]
    base_name =base_name.replace('?dl=0', "")
    base_name =base_name.replace('?dl=1', "")
    return re.sub('[^a-zA-Z0-9.]', '', base_name)

local_path = extract_filename(vid_file_url)
!wget -O $local_path $vid_file_url

--2023-01-25 19:04:28--  https://www.dropbox.com/s/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4 [following]
--2023-01-25 19:04:29--  https://www.dropbox.com/s/dl/q2gu4ik4h8zltpo/President%20Kennedy%27s%201962%20Speech%20on%20the%20US%20Space%20Program%20%20C-SPAN%20Classroom.mp4
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfa85c90c9c256528d467ec32d7.dl.dropboxusercontent.com/cd/0/get/B1Nt1Grfuly0NCTK3Xw2n6JK64iiDBCHJElgQniqdPuEu6WUyKw6q5ajw2-go-Rvz-YY8hM78It35Ku7wx3nb0gev4gCUB1vlPrzCgH

In [ ]:
import vid2cleantxt

data folder is set to `/usr/local/lib/python3.8/dist-packages/neuspell/../data` script


## print out the docstring for API usage

- `vid2cleantxt.transcribe.transcribe_dir` is the primary function at time of writing
- colab will also display this upon hovering over the function

In [ ]:
import inspect 

deets =inspect.getfullargspec(vid2cleantxt.transcribe.transcribe_dir)
deets._asdict()

{'args': ['input_dir',
  'chunk_length',
  'model_id',
  'basic_spelling',
  'move_comp',
  'join_text',
  'print_memory_usage',
  'verbose'],
 'varargs': None,
 'varkw': None,
 'defaults': (30, 'openai/whisper-base.en', False, False, False, False, False),
 'kwonlyargs': [],
 'kwonlydefaults': None,
 'annotations': {'input_dir': str, 'chunk_length': int, 'model_id': str}}

## transcribe a video


- passing `"."` transcribes all videos in the current working directory



In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
text_output, metadata_output = vid2cleantxt.transcribe.transcribe_dir(
    input_dir=".", # notebook current working directory
    model_id='openai/whisper-small.en',
    chunk_length=30,
    join_text=True, # merge all transcribed text to one line (for demo outputs)
)



Loading models @ Jan-25-2023_-19-05-03 - may take some time...
if RT seems excessive, try --verbose flag or checking logfile


transcribing...:   0%|          | 0/1 [00:00<?, ?it/s]

Creating .wav audio clips:   0%|          | 0/8 [00:00<?, ?it/s]

Transcribing video:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


SC_pipeline - transcribed audio:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# final transcription files are saved to this folder
text_output

'/content/v2clntxt_transcriptions/results_SC_pipeline'

In [ ]:
# final metadata files are saved to this folder
metadata_output

'/content/v2clntxt_transc_metadata'

In [ ]:
#@markdown display outputs
from pathlib import Path
output_dir = Path(text_output)
files = [f for f in output_dir.iterdir() if f.suffix == '.txt']
with open(files[0], 'r') as f:
    text = f.read()
text

'Now look into space to the moon and to the planets beyond and we have vowed that we shall not see it governed by a hostile flag of conquest but by a banner of freedom and peace we have vowed that we shall not see space filled with weapons of mass destruction but with instruments of knowledge and understanding yet the vow of. Of. To. Of. The. In outer space as yet its hazards are hostile to us all its conquest deserves the best of all mankind and its opportunity for peaceful cooperation may never come again but why some say the moon why choose this as our goal and they may well ask why climb the highest mountain why 35 years ago fly the atlantic why does. We choose to go to the moon we choose to go to the moon we choose to go to the moon and do the other things not because they are easy but because they are hard because that goal will serve to organize and measure the best of our energies and skills because that challenge is one of the most important goals of our lives. It is for these

In [ ]:
outpath = str(files[0])
outpath

'/content/v2clntxt_transcriptions/results_SC_pipeline/president_20_kennedy_27_s_20196220_speech_20_on_20_the_20_us_20_space_20_program_2020_c_span_NSC_SBD.txt'

## export outputs from Colab

- for summarization/other analytics, or printing and hanging on your wall
- if you had transcribed multiple files, you can zip and download the exports via


```python
from google.colab import files
archive_name = "my_transcriptions.zip"
# zip the returned `text_output` from vid2cleantxt
!zip -r -9 -j $archive_name $text_output
files.download(archive_name)
# wait for colab to download
```


for some examples of summarization, see the [textsum](https://github.com/pszemraj/textsum) python package, which is a utility for easily running summarization on strings/files with models such as [this one](https://huggingface.co/pszemraj/long-t5-tglobal-base-16384-book-summary) 

---



In [ ]:
from google.colab import files

# uncomment & run to download
# files.download(outpath)